In [2]:
import datetime
import polars as pl
import dateparser
import plotly.express as px
import re
import numpy as np


In [3]:

file = "/home/zeke/Downloads/FamilyTimeline - Sheet1 (4).csv"

# with open(file) as f:
#     lines = f.readlines()

# lines = filter(lambda l: len(l) > 4, lines)
# splitted = [re.split(r'(?:(?:\ ){2,})?(?:\t|(?:\ ){2,})', line, maxsplit=1) for line in lines]
# print(list(filter(lambda l: len(l) < 2, splitted)))
raw = pl.read_csv(file)
# df = pl.DataFrame(splitted).transpose()
df = raw.select(
    start=(pl.col('date')
        .str.replace(r'(.+)\-.+', '$1')
        # If has a season, but doesn't have a - in it
        .str.replace(r'(?i)^(?:[^\-])*(spring)(?:[^\-])*$', 'march')
        .str.replace(r'(?i)^(?:[^\-])*(fall)(?:[^\-])*$', 'september')
        .str.replace(r'(?i)^(?:[^\-])*(winter)(?:[^\-])*$', 'december')
        .str.replace(r'(?i)^(?:[^\-])*(summer)(?:[^\-])*$', 'june')
        # is a year-year, only capturing the first year
        # .str.replace(r'(?i)((?:\d){4})(?:\s+)?\-(?:\s+)?(?:\d){4}', '$1')
        .map_elements(lambda r: dateparser.parse(r, settings={"PREFER_MONTH_OF_YEAR": "first"}))
    ),
    end=(pl.col('date')
        .str.replace(r'.+\-(.+)', '$1')
        # If has a season, but doesn't have a - in it
        .str.replace(r'(?i)^(?:[^\-])*(spring)(?:[^\-])*$', 'march')
        .str.replace(r'(?i)^(?:[^\-])*(fall)(?:[^\-])*$', 'september')
        .str.replace(r'(?i)^(?:[^\-])*(winter)(?:[^\-])*$', 'december')
        .str.replace(r'(?i)^(?:[^\-])*(summer)(?:[^\-])*$', 'june')
        # is a year-year, only capturing the second year
        # .str.replace(r'(?i)(?:\d){4}(?:\s+)?\-(?:\s+)?((?:\d){4})', '$1')
        .map_elements(lambda r: dateparser.parse(r, settings={"PREFER_MONTH_OF_YEAR": "last"}))
    ),
    entry=pl.col('date'),
    event=pl.col('event'),
).filter(~pl.col('event').is_null())
# print(df)

# df.write_excel("C:\\Users\\donna\\OneDrive\\Documents\\From Clickfree\\Family History\\histories\\FamilyTimeline.xlsx")
# df = pl.read_excel("C:\\Users\\donna\\OneDrive\\Documents\\From Clickfree\\Family History\\histories\\FamilyTimeline.xlsx")

# df = df.drop_nulls('date')

# df = df.with_columns(date2=pl.col('date') + pl.duration(days=20), y=pl.lit(np.random.rand(df.height)))
# print(df)
# fig = px.timeline(data_frame=df, x_start='date', x_end='date2', text='event', y='y')
# fig.show()


sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


In [4]:
df = (df.with_columns(end=pl.when(pl.col('start') == pl.col('end'))
    .then(pl.col('start') + pl.duration(days=10))
    .otherwise(pl.col('end')))
)

In [5]:
from datetime import timedelta

wrap = 50
# df['start'].dt.round(timedelta(365*10))
df = df.with_columns(
    decade=df['start'].dt.round('10y'),
    # year_small=df['start'].dt.year().cast(str).str.slice(-1, 1),
    adj_start=pl.date(
        day=pl.when(pl.col('start').dt.month().cast(pl.Int64).eq(2) & pl.col('start').dt.day().cast(pl.Int64).eq(29)).then(28).otherwise(pl.col('start').dt.day()),
        month=df['start'].dt.month(),
        year=df['start'].dt.year().cast(str).str.slice(-1, 1).cast(int) + 1
    ),
    adj_end=pl.date(
        day=pl.when(pl.col('end').dt.month().cast(pl.Int64).eq(2) & pl.col('end').dt.day().cast(pl.Int64).eq(29)).then(28).otherwise(pl.col('end').dt.day()),
        month=df['end'].dt.month(),
        year=df['end'].dt.year().cast(str).str.slice(-1, 1).cast(int) + 1
    ),
    event_short=pl.col('event').map_elements(lambda txt: txt[:wrap] + '...' if len(txt) > wrap else txt)
    # adj_end=pl.date(
    #     day=pl.lit(28) if df['end'].dt.month() == 2 & df['end'].dt.day() == 29 else df['end'].dt.day(),
    #     month=df['end'].dt.month(),
    #     year=df['end'].dt.year().cast(str).str.slice(-1, 1).cast(int) + 1
    # ),
)
df

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


start,end,entry,event,decade,adj_start,adj_end,event_short
datetime[μs],datetime[μs],str,str,datetime[μs],date,date,str
1957-02-08 00:00:00,1957-02-18 00:00:00,"""1957 Feb 8""","""Dee & Donna married in Idaho F…",1960-01-01 00:00:00,0008-02-08,0008-02-18,"""Dee & Donna married in Idaho F…"
1957-02-09 00:00:00,1957-02-19 00:00:00,"""1957 Feb""","""Moved in Lewisville little hou…",1960-01-01 00:00:00,0008-02-09,0008-02-19,"""Moved in Lewisville little hou…"
1957-03-09 00:00:00,1957-03-19 00:00:00,"""1957 Mar""","""Moved to Terreton (the pig hou…",1960-01-01 00:00:00,0008-03-09,0008-03-19,"""Moved to Terreton (the pig hou…"
1957-04-09 00:00:00,1957-04-19 00:00:00,"""1957 Apr""","""Donna Relief Society Visiting …",1960-01-01 00:00:00,0008-04-09,0008-04-19,"""Donna Relief Society Visiting …"
2024-09-09 00:00:00,2024-09-19 00:00:00,"""1957 Fall""","""Donna called Relief Society Si…",2020-01-01 00:00:00,0005-09-09,0005-09-19,"""Donna called Relief Society Si…"
…,…,…,…,…,…,…,…
2024-05-29 00:00:00,2024-06-08 00:00:00,"""2024 May 29""","""Ellie Lynn graduated from Suga…",2020-01-01 00:00:00,0005-05-29,0005-06-08,"""Ellie Lynn graduated from Suga…"
2024-01-09 00:00:00,2024-05-25 00:00:00,"""2024-25 May""","""Hampton chosen for Student Lea…",2020-01-01 00:00:00,0005-01-09,0005-05-25,"""Hampton chosen for Student Lea…"
2024-04-24 00:00:00,2024-05-04 00:00:00,"""2024 Apr 24""","""Gideon Hardy was born to Ammon…",2020-01-01 00:00:00,0005-04-24,0005-05-04,"""Gideon Hardy was born to Ammon…"


In [15]:
from itertools import cycle

amt = 16
var = list(range(-amt, amt))
var.reverse()
var.remove(0)
c = cycle(var)
df = df.with_columns(y=pl.lit(np.array([next(c) for _ in range(df.height)])))

fig = px.timeline(df,
    x_start='adj_start',
    x_end='adj_end',
    text='event_short',
    y='y',
    title='Thueson Family Timeline (works best on desktop) -- Hover over an event to see more information                     <a href="https://smartycope.github.io/thueson-reunion/">Back</a>',
    facet_row='decade',
    # This doesn't work
    # category_orders={'decade': list((df['decade'].unique().to_list()))},
    height=4000,
    width=6000,
    hover_data={
        'y': False,
        'event': False,
        'entry': True,
        'adj_start': False,
        'adj_end': False,
        'event_short': False,
        'decade': False
    },
    hover_name='event',
    range_x=(datetime.date(1, 1, 1), datetime.date(11, 3, 31)),
    range_y=(-(amt+1), amt+1),
    labels={'entry': "When"}
)

fig.update_traces(marker=dict(line=dict(width=0)), selector=dict(type='bar'))
fig.update_traces(hoverinfo='skip', selector=dict(type='bar'))
fig.update_layout(hovermode='x')

# This is terrible, but it works
i = 0
def get_decade():
    global i
    i += 1
    return df['decade'].unique()[i-1]
fig.for_each_yaxis(lambda y: y.update(dict(
        tickmode='array',
        tickvals=[],
        showticklabels=False,
        title=str(get_decade())[:4] + "'s",
        # anchor='free',
        # position=.5,
    ),))

for idx, da in enumerate(zip(df['decade'].unique(), fig.layout.annotations)):
    decade, annotation = da
    facet = f"y{idx+1}" if idx > 0 else "y"
    annotation.text = str(decade)[:4] + "'s"

    fig.add_shape(
        type='line',
        y0=0,
        x0=min(df['adj_start']),# - pl.duration(weeks=520),
        y1=0,
        x1=max(df['adj_end']),# + pl.duration(weeks=520),
        line=dict(color='black', width=1),
        xref='x',
        yref=facet,
    )

fig

In [16]:
with open('familyTimeline.html', 'w') as f:
    f.write(fig.to_html())